In [77]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import timm
import numpy as np

In [78]:
seed_val = 420
torch.manual_seed(seed_val)

In [79]:
# functions for logging the misclassified images in tensorboard
def denormalize_image(tensor, mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
    """Denormalizes a tensor image by applying the inverse of the normalization transform."""
    # Reshape mean and std to match the (C, H, W) shape of the tensor
    mean = torch.tensor(mean).view(3, 1, 1).to(tensor.device)  # Match device as well
    std = torch.tensor(std).view(3, 1, 1).to(tensor.device)
    denormalized = tensor * std + mean
    return denormalized.clamp(0, 1)

def add_labels_to_image(image, true_label, pred_label):
    """Add true and predicted labels to the image."""
    # Convert the image to uint8 if it's in float format
    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)

    # Convert from RGB to BGR for OpenCV
    bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Prepare text without brackets
    text = f'True: {int(true_label)}, Pred: {int(pred_label)}'
    cv2.putText(bgr_image, text, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Convert back to RGB before returning
    return cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

def log_misclassified_images(misclassified_images, misclassified_preds, misclassified_true, writer):
    print("Logging misclassified images...")
    print(len(misclassified_images))
    labeled_images = []

    for i, img in enumerate(misclassified_images):
        img = denormalize_image(img)  # Denormalize the image
        img_np = img.permute(1, 2, 0).numpy()  # Convert (C, H, W) -> (H, W, C)
      

        # Ensure the image is in the right format for TensorBoard
        if img_np.shape[-1] == 1:  # Check if the image is grayscale
            img_np = img_np.squeeze(axis=2)  # Remove the channel dimension if it is 1
        elif img_np.shape[0] == 1:  # If the shape is (1, H, W)
            img_np = img_np.squeeze(axis=0)  # Remove the batch dimension
        img_np = np.clip(img_np, 0, 1)  # Ensure the values are between 0 and 1
        img_np = (img_np * 255).astype(np.uint8)     # Scale to 0-255

        true_label = misclassified_true[i]
        pred_label = misclassified_preds[i]

        labeled_image = add_labels_to_image(img_np, true_label, pred_label)
        # labeled_image = labeled_image.transpose(2, 0, 1)
        labeled_images.append(labeled_image)

    # Log image to TensorBoard
    writer.add_images(
        f'Misclassified',
        np.array(labeled_images),
        dataformats='NHWC'
    )

In [84]:
def test_on_folder(model, folder_path, device='cuda', transform=None, writer=None, true_label=None):
    """
    Apply a trained model to all images in a folder and calculate metrics.
    
    Args:
        model: Trained PyTorch model.
        folder_path: Path to the folder containing images.
        device: Device to run inference on ('cuda' or 'cpu').
        transform: Transformations to apply to images.
        writer: TensorBoard SummaryWriter for logging.
        true_label: Optional, the expected true label (used for calculating metrics).
    
    Returns:
        predictions: List of predicted classes.
        metrics: Dictionary with accuracy, recall, precision, F1-score (if true_label is provided).
    """
    # Move model to the appropriate device and set to eval mode
    model.to(device)
    model.eval()

    pred_list = []
    true_list = []
    misclassified_images = []
    misclassified_preds = []
    misclassified_true = []

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp')
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)]

    print(f"Processing {len(files)} images in {folder_path}.")
    pbar = tqdm(total=len(files), desc="Processing images")

    for idx, file_name in enumerate(files):
        img_path = os.path.join(folder_path, file_name)

        # Open and preprocess the image
        img = Image.open(img_path).convert("RGB")  # Ensure 3 channels
        if transform:
            img = transform(img)
        
        img_tensor = img.unsqueeze(0).to(device)  # Add batch dimension

        # Perform inference
        with torch.no_grad():
            output = model(img_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()

        pred_list.append(predicted_class)

        # Handle true labels and misclassifications
        if true_label is not None:
            true_list.append(true_label)
            if predicted_class != true_label:
                misclassified_images.append(img.cpu())
                misclassified_preds.append(predicted_class)
                misclassified_true.append(true_label)
        
        # Log to TensorBoard
        if writer:
            writer.add_scalar(f'Predictions/Probability/Image_{idx}', probabilities.max().item(), idx)
            writer.add_text(f'Predictions/Class/Image_{idx}', f"Image: {file_name}, Predicted: {predicted_class}, True: {true_label}", idx)

        pbar.update(1)

    pbar.close()

    # Calculate metrics if true labels are available
    metrics = {}
    if true_list:
        accuracy = 100. * sum(1 for p, t in zip(pred_list, true_list) if p == t) / len(true_list)

        metrics = {
            'accuracy': accuracy
        }

        # Log metrics to TensorBoard
        if writer:
            for metric, value in metrics.items():
                writer.add_scalar(f'Metrics/{metric}', value)

        # print(f"\nMetrics - {folder_path}")
        # for metric, value in metrics.items():
        #     print(f"{metric.capitalize()}: {value:.2f}%")

        # Log misclassified images if writer is provided
        if writer:
            log_misclassified_images(misclassified_images[:250], misclassified_preds[:250], misclassified_true[:250], writer)

    # Return predictions and metrics
    return pred_list, metrics


In [81]:
def process_multiple_folders(model, base_path, folder_list, device='cuda', transform=None, true_labels=None):
    """
    Process multiple folders using the given model and log results.

    Args:
        model: Trained PyTorch model.
        base_path: Base directory containing folders.
        folder_list: List of folder names to process.
        device: Device to run inference on ('cuda' or 'cpu').
        transform: Transformations to apply to images.
        true_labels: Dictionary mapping folder names to their true labels.

    Returns:
        results: Dictionary with folder names as keys and metrics as values.
    """
    results = {}
    for folder in folder_list:
        folder_path = os.path.join(base_path, folder)
        writer_path = f"runs/processed_test_set/{folder}"
        os.makedirs(writer_path, exist_ok=True)

        writer = SummaryWriter(writer_path)

        # Get true label for the folder
        true_label = true_labels.get(folder, 0)  # Default to 0 if not specified

        _, metrics = test_on_folder(
            model, folder_path, device=device, transform=transform, writer=writer, true_label=true_label
        )
        writer.close()

        results[folder] = metrics

    return results


In [85]:
# Base directory and folders to process
base_path = "../data/processed_test_set"
folders = ["grayscale_generated", "grayscale_real", "high_freq_generated", "high_freq_real", "low_freq_generated", 
           "low_freq_real", "resized_generated","resized_real"] 
true_labels = {
    "grayscale_generated": 0,
    "grayscale_real": 1,
    "high_freq_generated": 0,
    "high_freq_real": 1,
    "low_freq_generated": 0,
    "low_freq_real": 1,
    "resized_generated": 0,
    "resized_real": 1
}

folders = ["high_freq_generated"] 

# Load the trained model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = timm.create_model('convnext_base', pretrained=False, num_classes=2)
model_path = 'saved_models/convnext_model/RealArt_vs_FakeArt_convnext_base.pt'
model.load_state_dict(torch.load(model_path, map_location=device))

# Define transformations
test_data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Process all folders
results = process_multiple_folders(
    model, base_path, folders, device=device, transform=test_data_transforms, true_labels=true_labels
)

# Print summary of results
for folder, metrics in results.items():
    print(f"Results for {folder}: {metrics}")

Processing 3315 images in ../data/processed_test_set/high_freq_generated.








































































































































































































































































































































































































































































































































































































































































































Processing images: 100%|██████████| 3315/3315 [00:25<00:00, 128.13it/s]


Logging misclassified images...
250
Results for high_freq_generated: {'accuracy': 25.55052790346908}
